12/26/2017  carolyn.parkinson@gmail.com

This script creates and executes AFNI scripts to preprocess time series data
for a list of subjects. It takes as input:

    1) A list of subject names to iterate through (here, 'subjects.json')
    2) Nifti or AFNI files for each functional run for each subject
    3) Each subject's high-resolution anatomical scan
    4) Each subject's anatomical segmentation file ('aseg'), which was created
    with Freesurfer's recon-all command and converted to AFNI format with the
    AFNI command @SUMA_Make_Spec_FS.

    In preprocessing, the aseg file is used to obtain anatomical masks of the
    ventricles and white matter to generate global and local nuisance regressors.
    See relevant AFNI documentation for further details on how these are defined,
    and on the various options that are specified (but feel free to ask me
    if anything is unclear!):
    https://afni.nimh.nih.gov/pub/dist/doc/program_help/afni_restproc.py.html

In [2]:
#!/usr/bin/env python

# import relevant python modules
import os
import shutil
import subprocess
import json
import glob


In [3]:
mypath = ('/Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts')

In [4]:
subj_list = ["sub-289"]

In [5]:
# define a function to execute shell commands
def sh(c):
    '''
    run shell commands
    '''
    subprocess.call(c, shell = True)


In [24]:
for subject in subj_list:

    # specify and enter directory containing this subject's nifti files
    nifti_dir = "{}/nifti/{}".format(mypath, subject)
    os.chdir(nifti_dir)
  
    # specify relevant anatomical files for this subject
        # I think that aseg_Alnd_Exp+orig is the aseg file in the original 3D space (not aligned to MNI)
        #hires = "{}/{}_SurfVol_Alnd_Exp+orig".format(nifti_dir, subject)
        #aseg = "{}/aseg_Alnd_Exp+orig".format(nifti_dir)
        
    hires = "{}/{}_SurfVol".format(nifti_dir, subject)
    aseg = "{}/aseg".format(nifti_dir)
    
    # create a dictionary of EPI files, where the keys are run titles ('epi1',
    # 'epi2,', etc.), and the values are run file names.
    my_epis = {}
    #tr_nums = [204, 276, 194, 147, 189, 108, 160]
    videos = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    for epi_num, num_trs in enumerate(videos, start=1):
        my_epis['epi{}'.format(epi_num)] = glob.glob('{}/*video_{}*'.format(nifti_dir, num_trs))[0]
        
    for epi_name, epi_file in my_epis.items():
        sh_cmd1 = ("""afni_restproc.py \
              -anat {hires}.nii \
              -epi {epi} \
              -prefix {epiname}_preprocessed \
              -script {epiname}_rest_proc_script.tcsh \
              -epi2anat \
              -uniformize \
              -anat_has_skull yes \
              -aseg {aseg}.nii \
              -dreg \
              -trcut 2  \
              -localnorm \
              -smoothfirst \
              -smoothrad 4 \
              -polort 3 \
              -despike on""").format(hires = hires, epi = epi_file, epiname = epi_name, aseg = aseg)
        print(sh_cmd1)

afni_restproc.py               -anat /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/sub-289_SurfVol.nii               -epi /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/video_8.nii               -prefix epi8_preprocessed               -script epi8_rest_proc_script.tcsh               -epi2anat               -uniformize               -anat_has_skull yes               -aseg /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg.nii               -dreg               -trcut 2                -localnorm               -smoothfirst               -smoothrad 4               -polort 3               -despike on
afni_restproc.py               -anat /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/sub-289_SurfVol.nii               -epi /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/video_9.nii               -prefix epi9_preprocessed               -script epi9_rest_proc_script.tcsh               -epi2anat               -uniformize 

In [22]:
print(sh_cmd1)

afni_restproc.py               -anat /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/sub-289_SurfVol.nii               -epi /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/video_3.nii               -prefix epi3_preprocessed               -script epi3_rest_proc_script.tcsh               -epi2anat               -uniformize               -anat_has_skull yes               -aseg /Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts/nifti/sub-289/aseg.nii               -dreg               -trcut 2                -localnorm               -smoothfirst               -smoothrad 4               -polort 3               -despike on


In [17]:
sh('source activate python27')